In [5]:
import pybaseball
import polars as pl
import boto3
import datetime as dt
import pandas as pd

In [2]:
start_dates = pl.date_range(start=dt.date(2024, 2, 15),
                            end=dt.date(2024, 11, 30), 
                            interval='1w',
                            eager=True)

In [ ]:
dates = pl.DataFrame({'start_date': pl.date_range(dt.date(2024, 2, 15),
                                                  dt.date(2024, 11, 30),
                                                  interval='1w',
                                                  eager=True)})
dates = dates.with_columns(
    pl.col('start_date').dt.offset_by('1w').alias('end_date')
)

# I know iter_rows is unwise. But this is only about 40 entries.
frames_list = [pybaseball.statcast(start_dt=str(i[0]),
                                   end_dt=str(i[1]),
                                   verbose=False) for i in dates.iter_rows()]
frames_list = [i for i in frames_list if i.shape[0] != 0]

In [13]:
df_big = pl.from_pandas(pd.concat(frames_list))

In [24]:
unique_months = df_big['game_date'].dt.month().unique()
month_dict = {k: df_big.filter(
    pl.col('game_date').dt.month() == k) for k in unique_months}

In [ ]:
# TODO: Put each month's data into s3